<a href="https://colab.research.google.com/github/VeerendarGoud/ML-Projects/blob/master/Pyspark_NLP_Topic_modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [2]:
#install java

!apt-get install -y openjdk-8-jdk-headless -qq > /dev/null

In [3]:
# Setup java environment vars
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'
os.environ['PATH'] = os.environ['JAVA_HOME'] + '/bin:' + os.environ['PATH']

In [4]:
!java -version

openjdk version "1.8.0_382"
OpenJDK Runtime Environment (build 1.8.0_382-8u382-ga-1~22.04.1-b05)
OpenJDK 64-Bit Server VM (build 25.382-b05, mixed mode)


In [5]:
!echo $JAVA_HOME

/usr/lib/jvm/java-8-openjdk-amd64


In [6]:
# Install pyspark
!pip install pyspark==3.3.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 MB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 17.6 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764004 sha256=44fd9c20a70f73d2277fd42f9a22c8a3954e117a089837e2a1f1e11722c08f61
  Stored in directory: /root/.cache/pip/wheels/81/9c/6c/d5200fcf351ffa39cbe09911e99703283624cd037df58070d9
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [7]:
# Install spark NLP
!pip install spark-nlp==5.0.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.1/502.1 kB 12.7 MB/s eta 0:00:00


In [8]:
# !pip uninstall spark-nlp pyspark -y

In [9]:
#Install nltk
!pip install nltk

In [9]:
# dwonload the data from here
# https://www.kaggle.com/datasets/eswarchandt/amazon-music-reviews

In [10]:
!unzip /content/archive.zip

Archive:  /content/archive.zip
  inflating: Musical_Instruments_5.json  
  inflating: Musical_instruments_reviews.csv  


In [11]:
import pyspark

In [12]:
pyspark.__version__

'3.3.0'

In [13]:
import sys

In [14]:
sys.version

'3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]'

In [15]:
import sparknlp


# sparknlp.version()



In [16]:
spark = sparknlp.start()

In [10]:
# from pyspark import SparkConf, SparkContext
# sc = SparkContext.getOrCreate()

In [17]:
from pyspark.sql import functions as F

# Load the data
data_path = '/content/Musical_instruments_reviews.csv'
data = spark.read.csv(data_path, header=True)

In [18]:
# data info
data.columns

['reviewerID',
 'asin',
 'reviewerName',
 'helpful',
 'reviewText',
 'overall',
 'summary',
 'unixReviewTime',
 'reviewTime']

In [19]:
text_col = 'reviewText'
review_text = data.select(text_col).filter(F.col(text_col).isNotNull())

In [20]:
review_text.limit(5).show(truncate=90)

+------------------------------------------------------------------------------------------+
|                                                                                reviewText|
+------------------------------------------------------------------------------------------+
|                                                                          that's just like|
|The product does exactly as it should and is quite affordable.I did not realized it was...|
|The primary job of this device is to block the breath that would otherwise produce a po...|
|Nice windscreen protects my MXL mic and prevents pops. Only thing is that the gooseneck...|
|This pop filter is great. It looks and performs like a studio filter. If you're recordi...|
+------------------------------------------------------------------------------------------+



### Spark NLP Pipeline

In [24]:
PerceptronModel.pretrained

pos_anc download started this may take some time.
Approximate size to download 3.9 MB
[OK!]


POS_dbb704204f6f

In [22]:
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline

In [16]:
#
from sparknlp.base import DocumentAssembler

document_assembler = DocumentAssembler() \
                        .setInputCol('reviewText') \
                        .setOutputCol('document')

In [17]:
# tokenizer

from sparknlp.annotator import Tokenizer

tokenizer = Tokenizer() \
            .setInputCols(['document']) \
            .setOutputCol('tokenized')

In [18]:
# Normalizer

from sparknlp.annotator import Normalizer

normalizer = Normalizer()\
            .setInputCols(['tokenized'])\
            .setOutputCol('normalized') \
            .setLowercase(True)

In [19]:
# Lemmatization

from sparknlp.annotator import LemmatizerModel

lemmatizer = LemmatizerModel.pretrained()\
            .setInputCols(['normalized']) \
            .setOutputCol('lemmatized')

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]


In [20]:
# Download stop words

import nltk

nltk.download('stopwords')

from nltk.corpus import stopwords

eng_stopwords = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [21]:
# StopWords cleaner

from sparknlp.annotator import StopWordsCleaner

stopwords_cleaner = StopWordsCleaner()\
                .setInputCols(['lemmatized']) \
                .setOutputCol('unigram') \
                .setStopWords(eng_stopwords)

In [22]:
# NGram
# In addition to unigrams, it is good to use n-grams for topic
# modelling as well since they help to better refine topics.

from sparknlp.annotator import NGramGenerator

ngrammer = NGramGenerator()\
            .setInputCols(['lemmatized'])\
            .setOutputCol('ngram') \
            .setN(3) \
            .setEnableCumulative(True)\
            .setDelimiter('_')

In [23]:
#POS
from sparknlp.annotator import PerceptronModel

pos_tagger = PerceptronModel.pretrained('pos_anc') \
            .setInputCols(['document', 'lemmatized']) \
            .setOutputCol('pos')

pos_anc download started this may take some time.
Approximate size to download 3.9 MB
[OK!]


In [32]:
from sparknlp.base import Finisher

finisher = Finisher() \
     .setInputCols(['unigram', 'ngram', 'pos'])

In [33]:
## Pipeline

from pyspark.ml import Pipeline

pipeline = Pipeline() \
            .setStages([
                document_assembler,
                 tokenizer,
                 normalizer,
                 lemmatizer,
                 stopwords_cleaner,
                 pos_tagger,
                 ngrammer,
                 finisher
            ])

In [34]:
processed_review = pipeline.fit(review_text).transform(review_text)

In [35]:
processed_review.limit(5).show()

+--------------------+--------------------+--------------------+--------------------+
|          reviewText|    finished_unigram|      finished_ngram|        finished_pos|
+--------------------+--------------------+--------------------+--------------------+
|    that's just like|              [like]|[that, just, like...|        [IN, RB, IN]|
|The product does ...|[product, exactly...|[the, product, do...|[DT, NN, VBP, RB,...|
|The primary job o...|[primary, job, de...|[the, primary, jo...|[DT, JJ, NN, IN, ...|
|Nice windscreen p...|[nice, windscreen...|[nice, windscreen...|[JJ, NN, NN, NNP,...|
|This pop filter i...|[pop, filter, gre...|[this, pop, filte...|[DT, NN, NN, VB, ...|
+--------------------+--------------------+--------------------+--------------------+

